# Structural-Based extractive summarization

以往的抽取式摘要算法：
1. 没有有考虑到**文章结构**对于关键词的影响 (只是使用启发式算法)
2. 使用外部语料作为重要度的来源 (如CNN等语料训练tf-idf)
3. 没有考虑到词之间的多种语义关系（如wordNet, word embedding）


本文内容
1. 本文首先验证了，在domain-specific的摘要系统中，无法获取准确的词重要度，依据文章内部的资源进行重要度抽取（与两个假定的冲突），并通过t-test检验这种差距的显著性
2. 使用多种方法基于文章结构测定词的重要性
3. 基于多种语义方式构建多种keywords之间的关系，并使用rank公式来找出关键词
4. 利用sentence shortening技术，无需background corpus和POS等词性标注就能很好地完成文本摘要工作
5. 这种重要性公式对于抽取文章维度、提供文章语义也有帮助。

实验对比
1. 与LexRank, TextRank, Bert-centroid based 对比
2. 使用TokenToMe进行分词抽取
3. 使用词形还原后的
4. 去除停用词
5. 与RST系统对比
6. 与WordNet系统对比
7. 如果只在introduction和conclusion中进行摘要和全文摘要的对比

结果展示
1. 展示一个抽取出来的摘要
2. 展示抽取出来的关键词

图表
1. 证明关键词比例的
2. 显示四种方式在各种ROUGE评分下的PR曲线



### 导入依赖


In [8]:
%load_ext autoreload
%autoreload 2
from code.data_loader import Corpus, tokenize, segment
import matplotlib.pyplot as plt
from metrics import rouge_score
from pathlib import Path
from functools import partial

tokenizer_ = partial(tokenize, remove_stop=True)
corpus = Corpus(tokenizer_)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
item = corpus.items[0]
all_sentences = segment(item.introduction + item.sections + item.conclusion)
fe_sentences = segment(item.introduction + item.conclusion)

truth = tokenizer_(item.abstract)

### 四种方法的PR曲线

1. tf
2. tf-idf
3. chi2
4. mutual information
5. log likelihood

In [18]:
def prec_recall(sent_toks, metric="rouge-1", desired_length=60):
    choices = []
    for toks in sent_toks:
        choices += toks
        if len(choices) >= desired_length:
            break
    summary = " ".join(choices)
    score = rouge_score(truth, summary)[0][metric]
    
    return score["p"], score["r"], score["f"]

def prec_recall_list(sent_toks, metric="rouge-1", mean=False):
    ps, rs = [], []
    for desired_length in range(5, 300, 10):
        p, r, f = prec_recall(sent_toks, metric, desired_length)
        ps.append(p)
        rs.append(r)
        
    return ps, rs

prec_recall_list(item.summarize("chi2"))

fig = plt.figure(figsize=(20, 10))

x_lim = {
    "rouge-1": [0, 0.7],
    "rouge-2": [0, 0.2],
    "rouge-l": [0, 0.6],
}

y_lim = {
    "rouge-1": [0, 0.4],
    "rouge-2": [0, 0.2],
    "rouge-l": [0, 0.3],
}

for i, metric in enumerate(["rouge-1", "rouge-2", "rouge-l"], 1):
    axis = fig.add_subplot(2, 2, i)
    axis.set_title(metric)
    for label, color in zip(["tf", "idf", "chi2", "gi", "llr"], 
                           ["green", "darkorange", "brown", "blue", "dark"]):
        ps, rs = prec_recall_list(item.summarize(label), metric)
        axis.plot(rs, ps, label=label, color=color)
        
    p, r = lex_pr(metric)
    axis.plot(r, p, label="lex", color="red")
    axis.legend()
    axis.set_xlim(x_lim[metric])
    axis.set_ylim(y_lim[metric])
    

plt.show()

ValueError: Invalid RGBA argument: 'dark'

<Figure size 1440x720 with 3 Axes>

### LexRank

In [13]:
from lexrank import STOPWORDS, LexRank
documents = []
documents_dir = Path('../bbc/tech')

for file_path in documents_dir.glob('*.txt'):
    with file_path.open(mode='rt', encoding='utf-8') as fp:
        documents.append(fp.readlines())
lxr = LexRank(documents, stopwords=STOPWORDS['en'])

In [14]:
lex_all_summary = lxr.get_summary(all_sentences, summary_size=5, threshold=.1)
rouge_score(truth, " ".join(lex_all_summary))

[{'rouge-1': {'f': 0.13432835379817348, 'p': 0.1, 'r': 0.20454545454545456},
  'rouge-2': {'f': 0.023809519832767103,
   'p': 0.01639344262295082,
   'r': 0.043478260869565216},
  'rouge-l': {'f': 0.11093806805283159, 'p': 0.1, 'r': 0.20454545454545456}}]

In [15]:
lex_fe_summary = lxr.get_summary(fe_sentences, summary_size=20, threshold=.1)
rouge_score(truth, " ".join(lex_fe_summary))

[{'rouge-1': {'f': 0.16374268781642218,
   'p': 0.09395973154362416,
   'r': 0.6363636363636364},
  'rouge-2': {'f': 0.021089629445300805,
   'p': 0.011472275334608031,
   'r': 0.13043478260869565},
  'rouge-l': {'f': 0.08544650043361231,
   'p': 0.08389261744966443,
   'r': 0.5681818181818182}}]

In [16]:
def lex_pr(metric):
    p, r = prec_recall_list([tokenizer_(summ, join=False) for summ in lex_fe_summary], metric)
    return p, r

In [ ]:
def llr_f(inputs):
    X = vectorizer.fit_transform(inputs.data).toarray()
    Y = np.array(inputs.target)
    Y = LabelBinarizer().fit_transform(Y)
    if Y.shape[1] == 1:
        Y = np.append(1 - Y, Y, axis=1)

    col = X / np.sum(a, axis=0);

    

In [416]:
X = vectorizer.fit_transform(inputs.data).toarray()
Y = np.array(inputs.target)
Y = LabelBinarizer().fit_transform(Y)
if Y.shape[1] == 1:
    Y = np.append(1 - Y, Y, axis=1)
freq = np.dot(Y.T, X)

In [417]:
col_prob = freq / np.sum(freq, axis=0)
row_prob = freq / np.sum(freq, axis=1).reshape(-1, 1)
N = freq.sum()
row_sum = np.sum(freq, axis=1).reshape(-1, 1)

def log(prob):
    prob[prob == 0] = prob[prob == 0] + 1 / N
    return np.log(prob)


row_entropy = np.sum(freq * log(row_prob) + (row_sum - freq) * log(1 - row_prob), axis=0)
col_entropy = np.sum(freq * log(col_prob), axis=0) + np.sum((row_sum - freq) * log((row_sum - freq) / np.sum(row_sum - freq, axis=0)), axis=0)
mat_entropy = np.sum(freq * log(freq / N), axis=0) + np.sum((row_sum - freq) * log((row_sum - freq) / N), axis=0)

llrv = - 2 * (mat_entropy - row_entropy - col_entropy)
return llrv

In [384]:
row_sum

array([[ 6],
       [24]])

In [382]:
col

array([[0. , 0.5, 1. , ..., 1. , 1. , 1. ],
       [1. , 0.5, 0. , ..., 0. , 0. , 0. ]])

In [383]:
row

array([[0.00108342, 0.00216685, 0.00325027],
       [0.05501618, 0.00970874, 0.01294498]])

In [ ]:
np.dot(Y.T, X)

In [326]:
def logLikelihoodRatio(k11, k12, k21, k22):
        rowEntropy = entropy(k11, k12) + entropy(k21, k22);
        columnEntropy = entropy(k11, k21) + entropy(k12, k22);
        matrixEntropy = entropy(k11, k12, k21, k22);
        return 2 * (matrixEntropy - rowEntropy - columnEntropy);

def entropy(*elements):
        s = sum(elements)
        result = 0.0;
    
        for x in elements:
            # int zeroFlag = (x == 0 ? 1 : 0);
            result += x * log((x + 0) / s);
        
        return -result;

In [394]:
logLikelihoodRatio(1, 5, 2, 7)

0.07062661671220027

In [356]:
a = np.array([
    [1, 2, 3],
    [17, 3, 4]
])

In [357]:
col = a / np.sum(a, axis=0);
col

array([[0.05555556, 0.4       , 0.42857143],
       [0.94444444, 0.6       , 0.57142857]])

In [358]:
row = a / np.sum(a, axis=1).reshape(-1, 1)
row

array([[0.16666667, 0.33333333, 0.5       ],
       [0.70833333, 0.125     , 0.16666667]])

In [359]:
N = a.sum()

In [360]:
re = np.sum(a * log(row) + (row_sum - a) * log(1 - row), axis=0) ;
re

array([-17.19066129, -12.86156888, -14.9723521 ])

In [336]:
col_sum = np.sum(a, axis=0).reshape(1, -1)

In [337]:
col_sum - a

array([[17,  3,  4],
       [ 1,  2,  3]])

In [338]:
a

array([[ 1,  2,  3],
       [17,  3,  4]])

In [339]:
row_sum - a

array([[ 5,  4,  3],
       [-8,  6,  5]])

In [340]:
row

array([[0.16666667, 0.33333333, 0.5       ],
       [0.70833333, 0.125     , 0.16666667]])

In [341]:
ce = np.sum(a * log(col), axis=0) + np.sum((row_sum - a) * log((row_sum - a) / np.sum(row_sum - a, axis=0)), axis=0);
ce

/home/lee/.conda/envs/ccf/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([         nan, -10.09517501, -10.07286264])

In [342]:
me = np.sum(a * log(a / N), axis=0) + np.sum((row_sum - a) * log((row_sum - a) / N), axis=0);
me

/home/lee/.conda/envs/ccf/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([         nan, -30.04009524, -30.83391999])

In [343]:
2 * (me - re - ce)

array([         nan, -18.17264448, -17.04705721])

In [344]:
1 * log(1 / 15) + 2 * log(2 / 15) + 5 * log(5 / 15) + 7 * log(7 / 15)

-17.565898049855566

In [345]:
log(1 / 3) + 2 * log(2 / 3) + 5 * log(5 / 12) + 7 * log(7 / 12)

-10.059861696782747

In [346]:
log(1 / 3)

-1.0986122886681098

In [347]:
row

array([[0.16666667, 0.33333333, 0.5       ],
       [0.70833333, 0.125     , 0.16666667]])

In [348]:
a * log(row)

array([[-1.79175947, -2.19722458, -2.07944154],
       [-5.86228827, -6.23832463, -7.16703788]])

In [349]:
log(1 / 6) + 5 * log(5 / 6)

-2.703367253197828

In [350]:
row_sum = np.sum(a, axis=1).reshape(-1, 1)

In [351]:
log(1 / 6) + 5 * log(5 / 6) + 2 * log(2 / 9) + 7 * log(7 / 9)

-7.470723044716719

In [352]:
log(1 / 3) + 2 * log(2 / 3) + 5 * log(5 / 12) + 7 * log(7 / 12)

-10.059861696782747

In [353]:
N

30

In [354]:
from code.info import llr_f, chi2_f

In [355]:
dict(chi2_f(inputs))

{'1349837': 3.857142857142857,
 '1502': 1.0582010582010584,
 '1750': 0.2592592592592593,
 '1800': 0.2592592592592593,
 '1836': 1.0582010582010584,
 '1849': 0.5185185185185186,
 '2006': 0.2592592592592593,
 '383': 0.2592592592592593,
 '480': 0.2592592592592593,
 '528': 0.2592592592592593,
 'accuracy': 0.2980599647266314,
 'accuracybaseline': 0.2592592592592593,
 'active': 3.857142857142857,
 'adaptation': 4.407407407407407,
 'add': 0.5185185185185186,
 'adding': 0.2592592592592593,
 'again': 0.2592592592592593,
 'al': 0.5417989417989418,
 'algorithm': 1.0582010582010584,
 'algorithms': 3.857142857142857,
 'all': 0.2592592592592593,
 'alternative': 0.5961199294532628,
 'alternatives': 0.2592592592592593,
 'analysis': 3.895943562610229,
 'analytic': 0.2592592592592593,
 'analytically': 2.1164021164021167,
 'annotations': 1.0582010582010584,
 'appear': 0.2592592592592593,
 'applicability': 3.857142857142857,
 'application': 0.2592592592592593,
 'applied': 0.7777777777777778,
 'apply': 1.15

In [134]:
dict(llr_f(inputs))

{'1349837': 13.81750955863044,
 '1502': -0.0,
 '1750': -13.81750955863044,
 '1800': -13.81750955863044,
 '1836': -0.0,
 '1849': -13.81750955863044,
 '2006': -13.81750955863044,
 '383': -13.81750955863044,
 '480': -13.81750955863044,
 '528': -13.81750955863044,
 'accuracy': -1.3833010954077654,
 'accuracybaseline': -13.81750955863044,
 'active': 13.81750955863044,
 'adaptation': -0.7118957467189511,
 'add': -13.81750955863044,
 'adding': -13.81750955863044,
 'again': -13.81750955863044,
 'al': -1.6907952445714285,
 'algorithm': -0.0,
 'algorithms': 13.81750955863044,
 'all': -13.81750955863044,
 'alternative': -1.3833010954077654,
 'alternatives': -13.81750955863044,
 'analysis': 1.3833010954077654,
 'analytic': -13.81750955863044,
 'analytically': -0.0,
 'annotations': -0.0,
 'appear': -13.81750955863044,
 'applicability': 13.81750955863044,
 'application': -13.81750955863044,
 'applied': -13.81750955863044,
 'apply': -0.8092670144572764,
 'applying': -13.81750955863044,
 'approach': -

In [135]:
X = vectorizer.fit_transform(inputs.data).toarray()
Y = np.array(inputs.target)
Y = LabelBinarizer().fit_transform(Y)
if Y.shape[1] == 1:
    Y = np.append(1 - Y, Y, axis=1)

prob = np.dot(Y.T, X) / np.sum(X, axis=0) + 1e-3   # n_classes * n_features
prob = prob / np.sum(prob, axis=0) 
sign = -2 * (np.log(prob[0]) - np.log(prob[1]))


In [154]:
np.max(np.log(prob), axis=0).shape - np.min(np.log(prob), axis=0)

array([559.90975328, 553.69314718, 559.90975328, 559.90975328,
       553.69314718, 559.90975328, 559.90975328, 559.90975328,
       559.90975328, 559.90975328, 554.09761478, 559.90975328,
       559.90975328, 553.88687557, 559.90975328, 559.90975328,
       559.90975328, 554.20264302, 553.69314718, 559.90975328,
       559.90975328, 554.09761478, 559.90975328, 554.09761478,
       559.90975328, 553.69314718, 553.69314718, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 553.91579185,
       559.90975328, 554.09761478, 553.69314718, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 559.90975328,
       559.90975328, 553.84696525, 553.84696525, 559.90975328,
       559.90975328, 553.69314718, 559.90975328, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 559.90975328,
       559.90975328, 559.90975328, 559.90975328, 559.90

In [148]:
prob

array([[9.98003992e-04, 5.00000000e-01, 9.99001996e-01, ...,
        9.99001996e-01, 9.99001996e-01, 9.99001996e-01],
       [9.99001996e-01, 5.00000000e-01, 9.98003992e-04, ...,
        9.98003992e-04, 9.98003992e-04, 9.98003992e-04]])

In [142]:
prob

array([[9.98003992e-04, 5.00000000e-01, 9.99001996e-01, ...,
        9.99001996e-01, 9.99001996e-01, 9.99001996e-01],
       [9.99001996e-01, 5.00000000e-01, 9.98003992e-04, ...,
        9.98003992e-04, 9.98003992e-04, 9.98003992e-04]])

In [144]:
np.log(0.2) - np.log(0.6)

-1.0986122886681096

In [109]:
prob = prob / np.sum(X, axis=0) + 0.01

In [111]:
prob / np.sum(X, axis=0) 

array([[1.00000000e-02, 2.55000000e-01, 1.01000000e+00, ...,
        1.01000000e+00, 8.41666667e-02, 5.05000000e-01],
       [1.01000000e+00, 2.55000000e-01, 1.00000000e-02, ...,
        1.00000000e-02, 8.33333333e-04, 5.00000000e-03]])

In [120]:
np.log(0.01)

-4.605170185988091

In [82]:
inputs = item.transform()

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase=False)

from sklearn.preprocessing import LabelBinarizer
import numpy as np


inputs = item.transform()
X = vectorizer.fit_transform(inputs.data)
Y = np.array(inputs.target)
Y = LabelBinarizer().fit_transform(Y)

In [6]:
X.shape

(102, 553)

In [7]:
if Y.shape[1] == 1:
        Y = np.append(1 - Y, Y, axis=1)

In [8]:
Y.shape

(102, 2)

In [9]:
ob = np.dot(Y.T, X)

In [10]:
ob.shape

(2, 102)

In [15]:
if Y.shape[1] == 1:
        Y = np.append(1 - Y, Y, axis=1)

In [17]:
Y.shape

(102, 2)

In [52]:
prob = np.dot(Y.T, X.toarray())

In [53]:
prob.shape

(2, 553)

In [46]:
a = np.array([[1, 2, 3], [2, 3,  4
                         ]])

In [47]:
a

array([[1, 2, 3],
       [2, 3, 4]])

In [50]:
b = a / a.sum(axis=0)

array([[0.33333333, 0.4       , 0.42857143],
       [0.66666667, 0.6       , 0.57142857]])

In [56]:
b = a / a.sum(axis=0)

In [66]:
-2 * (np.log(b[0]) - np.log(b[1]))

array([1.38629436, 0.81093022, 0.57536414])

In [65]:
-2 * (np.log(0.33) - np.log(0.66))

1.3862943611198908

In [55]:
prob

array([[ 0,  1,  1, ...,  1, 12,  2],
       [ 1,  1,  0, ...,  0,  0,  0]], dtype=int64)

In [54]:
prob / prob.sum(axis=0)

array([[0. , 0.5, 1. , ..., 1. , 1. , 1. ],
       [1. , 0.5, 0. , ..., 0. , 0. , 0. ]])

In [51]:
prob

array([[ 0,  1,  1, ...,  1, 12,  2],
       [ 1,  1,  0, ...,  0,  0,  0]], dtype=int64)

In [44]:
prob.sum(axis=0).reshape(-1, 1)

array([[ 1],
       [ 2],
       [ 1],
       [ 1],
       [ 2],
       [ 2],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 3],
       [ 1],
       [ 1],
       [17],
       [ 2],
       [ 1],
       [ 1],
       [10],
       [ 2],
       [ 1],
       [ 1],
       [ 6],
       [ 1],
       [ 3],
       [ 1],
       [ 4],
       [ 2],
       [ 1],
       [ 1],
       [ 1],
       [ 3],
       [ 5],
       [ 1],
       [ 3],
       [ 4],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 7],
       [ 7],
       [ 1],
       [ 1],
       [ 2],
       [ 1],
       [ 2],
       [ 1],
       [ 1],
       [ 4],
       [ 2],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 3],
       [ 2],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 2],
       [ 5],
       [ 1],
       [ 1],
       [ 1],
       [ 2],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 3],
       [ 1],

In [32]:
prob.sum(axis=0).shape

(553,)

In [13]:
feature_count = X.sum(axis=0).reshape(-1, 1)
class_prob = Y.mean(axis=0).reshape(1, -1)

In [16]:
expected = np.dot(class_prob.T, feature_count)

ValueError: shapes (2,1) and (553,1) not aligned: 1 (dim 1) != 553 (dim 0)

In [257]:
Y.mean(axis=0)

array([0.79411765, 0.20588235])

In [ ]:
X.sum(axis)

In [250]:
X.shape

(102, 553)

In [228]:
Y

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Input(data=['unsupervised domain adaptation fundamental problem natural language processing hope apply systems datasets annotations', 'relevant labeled datasets stale comparison rapidly evolving social media writing styles increasing interest natural language processing historical texts', 'number approaches domain adaptation proposed tend emphasize bag-of-words features classification tasks sentiment analysis', 'consequently approaches rely instance large number active features fail exploit structured feature spaces characterize syntactic tasks sequence labeling parsing .as will substantial efficiency improvements designing domain adaptation methods learning structured feature spaces', 'build work deep learning community denoising autoencoders trained remove synthetic noise observed instances', 'autoencoder transform original feature space representation dependent individual feature robust domains', 'chen al', 'autoencoders learned noising process analytically marginalized idea spirit 

In [152]:
s = " ".join(map(lambda s: " ".join(s), item.summarize("chi2")[:]))
rouge_score(truth, s)

[{'rouge-1': {'f': 0.23140495570247935,
   'p': 0.1414141414141414,
   'r': 0.6363636363636364},
  'rouge-2': {'f': 0.043343650808116765,
   'p': 0.02527075812274368,
   'r': 0.15217391304347827},
  'rouge-l': {'f': 0.11009004563932587,
   'p': 0.10606060606060606,
   'r': 0.4772727272727273}}]

In [55]:
len(s.split(" "))

230

In [68]:
item.summarize("idf")[]

[['autoencoders',
  'learned',
  'noising',
  'process',
  'analytically',
  'marginalized',
  'idea',
  'spirit',
  'feature',
  'noising'],
 ['autoencoder',
  'transform',
  'original',
  'feature',
  'space',
  'representation',
  'dependent',
  'individual',
  'feature',
  'robust',
  'domains'],
 ['secondary',
  'contribution',
  'paper',
  'demonstrate',
  'applicability',
  'unsupervised',
  'domain',
  'adaptation',
  'syntactic',
  'analysis',
  'historical',
  'texts.denoising',
  'autoencoders',
  'provide',
  'intuitive',
  'solution',
  'domain',
  'adaptation',
  'transform',
  'features',
  'representation',
  'resistant',
  'noise',
  'characterize',
  'domain',
  'adaptation',
  'process'],
 ['exploit',
  'structure',
  'propose',
  'alternative',
  'noising',
  'techniques',
  '1',
  'feature',
  'scrambling',
  'randomly',
  'chooses',
  'feature',
  'template',
  'randomly',
  'selects',
  'alternative',
  'template',
  '2',
  'structured',
  'dropout',
  'randomly'

In [63]:
item.summarize("idf")[:-10]

[['supported', 'national', 'science', 'foundation', 'award', '1349837'],
 ['relevant',
  'labeled',
  'datasets',
  'stale',
  'comparison',
  'rapidly',
  'evolving',
  'social',
  'media',
  'writing',
  'styles',
  'increasing',
  'interest',
  'natural',
  'language',
  'processing',
  'historical',
  'texts'],
 ['step',
  'simplicity',
  'showing',
  'structured',
  'dropout',
  'marginalization',
  'easier',
  'obtaining',
  'dramatic',
  'speedups',
  'sacrificing',
  'accuracy',
  'reviewers',
  'feedback'],
 ['build',
  'work',
  'deep',
  'learning',
  'community',
  'denoising',
  'autoencoders',
  'trained',
  'remove',
  'synthetic',
  'noise',
  'observed',
  'instances'],
 ['example', 'part-of-speech', 'tagging', 'toutanova', 'al'],
 ['define',
  'feature',
  '“',
  'templates',
  '”',
  'current',
  'word',
  'previous',
  'word',
  'suffix',
  'current',
  'word'],
 ['consider',
  'feature',
  'structure.we',
  'apply',
  'ideas',
  'fine-grained',
  'part-of-speech',


In [71]:
s = " ".join(map(lambda s: " ".join(s), item.summarize("idf")[11:20]))
rouge_score(truth, s)

[{'rouge-1': {'f': 0.23437499548828133,
   'p': 0.17857142857142858,
   'r': 0.3409090909090909},
  'rouge-2': {'f': 0.0370370329705842,
   'p': 0.02586206896551724,
   'r': 0.06521739130434782},
  'rouge-l': {'f': 0.15917673715950592,
   'p': 0.14285714285714285,
   'r': 0.2727272727272727}}]

In [72]:
s = " ".join(map(lambda s: " ".join(s), item.summarize("idf")[:10]))
rouge_score(truth, s)

[{'rouge-1': {'f': 0.27544909791530714,
   'p': 0.18699186991869918,
   'r': 0.5227272727272727},
  'rouge-2': {'f': 0.0756756719392259,
   'p': 0.050359712230215826,
   'r': 0.15217391304347827},
  'rouge-l': {'f': 0.13153560723700325,
   'p': 0.12195121951219512,
   'r': 0.3409090909090909}}]

In [ ]:
s = " ".join(map(lambda s: " ".join(s), item.summarize("idf")[:10]))
rouge_score(truth, s)

In [73]:
summary_cont = lxr.get_summary(sentences, threshold=None)
print(summary_cont)

['we then describe two novel types of noise that are designed for structured feature spaces, and explain how they can be marginalized to efficiently compute 𝐖.in dropout noise, each feature is set to zero with probability p>.']
